# Data Processing for Subway Map

We use the open data [public transit timetable data set](https://daten.berlin.de/datensaetze/vbb-fahrplandaten-gtfs) for Berlin.
From this data set we extract (meta)data on subway stops as well as subway lines.

To run this notebook please make sure there is a ´data/´ directory filled as follows with the data available under the link above:
data/
├── routes.txt
├── stops.txt
├── stop_times.txt
└── trips.txt

## Tables
### Routes
relevant columns: `route_id, agency_id, stop_name`

get `route_id` for the 9 subway lines

### Trips
relevant columns: `route_id, trip_id`

get all trips on the route
join to routes table on `route_id`

### Stop times
relevant columns: trip_id, stop_id

get all 
join to routes/trips on `trip_id`

### Stops
relevant columns: `stop_id, stop_name, stop_lat, stop_lon`

get all stops
join to routes/trips/stop times on `stop_id`

This leaves us with a massive data set.
From this we extract a complete list of stations (in order) for each subway line.

In [11]:
import pandas as pd

In [12]:
# read tables
routes = pd.read_csv('./data/routes.txt')
trips = pd.read_csv('./data/trips.txt')
stop_times = pd.read_csv('./data/stop_times.txt')
stops = pd.read_csv('./data/stops.txt')

/home/marc/projects/ubahn_data/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/marc/projects/ubahn_data/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# select subway lines
subway_lines = ["U1", "U2", "U3", "U4", "U5", "U6", "U7", "U8", "U9"]
subway_routes = routes[routes['route_short_name'].isin(subway_lines)]
subway_routes

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,route_desc
937,17994_700,796,U2,NaN,700,NaN,NaN,NaN
952,17526_400,796,U9,NaN,400,NaN,NaN,NaN
953,17525_400,796,U8,NaN,400,NaN,NaN,NaN
954,17524_700,796,U8,NaN,700,NaN,NaN,NaN
955,17523_400,796,U7,NaN,400,NaN,NaN,NaN
956,17522_700,796,U7,NaN,700,NaN,NaN,NaN
957,17521_400,796,U6,NaN,400,NaN,NaN,NaN
958,17520_700,796,U6,NaN,700,NaN,NaN,NaN
959,17518_400,796,U5,NaN,400,NaN,NaN,NaN
960,17516_400,796,U4,NaN,400,NaN,NaN,NaN


# Get subway lines

In [14]:
one_way_routes = subway_routes[subway_routes['route_type'] == 400] # only one direction

# join tables
route_trips = pd.merge(one_way_routes, trips, on='route_id')
route_trips_stop_times = pd.merge(route_trips, stop_times, on='trip_id')
route_stops = pd.merge(route_trips_stop_times, stops, on='stop_id')

# this gets us a HUGE list of each individual stop on each subway trip

In [15]:
# we just need one COMPLETE trip with all stations (in the correct order) per subway line
full_trips_per_route = route_stops.groupby(['route_short_name', 'trip_id'])['stop_name'].count().sort_values().groupby(level=0).tail(1).keys()
# these are trip IDs for one full trip per subway line
trip_ids = [route[1] for route in full_trips_per_route]

In [16]:
# now we filter our data set
routes = route_stops[route_stops['trip_id'].isin(trip_ids)] \
  .sort_values(['route_short_name', 'stop_sequence'])

In [17]:
# we prepare the data as a json file to use in our web app
routes = routes[['route_id', 'route_short_name', 'stop_sequence', 'stop_id', 'parent_station', 'stop_name', 'stop_lat', 'stop_lon']]
routes.to_json('./stations.json', orient='records')

In [18]:
# in order to access the images from the web app, we need to rename each image with
# the corresponding stop ID (and an index for multiple pictures)
# to make this easier we create a spreadsheet
routes[['route_short_name', 'stop_name', 'stop_id']].to_excel('./station_ids.xlsx')